In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline
data=pd.read_excel('data/raw/Data_Set_(A+B).xlsx')

In [2]:
data

,I1,I2,I3,gx,gy,gz,ax,ay,az,V1real,V2real,V3real,N1,N2,N3,Type
0,1.126697,0.021116,0.927601,0.712941,-0.252941,1.081765,0.043529,0.003174,0.011661,-249.352941,4.764706,265.235294,-5070,30,5239,1
1,1.039215,0.015083,0.953243,0.229412,0.276471,0.485294,0.013672,-0.005572,0.012164,-260.470588,0.000000,263.647059,-5320,-9,5391,1
2,0.911011,0.004525,0.947210,1.170588,0.222941,0.664118,0.011704,0.003720,0.005744,-268.411765,0.000000,268.411765,-4829,6,4786,1
3,0.692308,1.156862,0.550528,1.165294,0.698235,-3.421765,0.001321,-0.010311,-0.001522,141.352941,-306.529412,162.000000,2670,-5662,2875,1
4,0.529412,1.024133,0.453997,0.808235,0.090000,-1.449412,0.006218,-0.010986,-0.006247,155.647059,-314.470588,169.941176,2841,-5824,2967,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,0.288889,0.449573,0.270085,0.153333,-2.288667,40.140000,-0.019906,0.003939,-0.000586,349.200000,340.200000,358.200000,5847,5663,5901,5
172,0.314530,0.331624,0.341880,0.096000,-2.154667,37.425333,-0.015479,0.000993,-0.009017,347.400000,343.800000,360.000000,6025,6018,6005,5
173,0.287179,0.377778,0.391453,-1.820667,-5.030000,82.963333,0.027490,0.046403,-0.012874,750.600000,739.800000,738.000000,12100,12073,12049,5
174,0.370940,0.427350,0.336752,-0.624000,-4.574000,83.285333,0.014160,-0.043896,0.003906,734.400000,745.200000,747.000000,12493,12493,12564,5


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   I1      176 non-null    float64
 1   I2      176 non-null    float64
 2   I3      176 non-null    float64
 3   gx      176 non-null    float64
 4   gy      176 non-null    float64
 5   gz      176 non-null    float64
 6   ax      176 non-null    float64
 7   ay      176 non-null    float64
 8   az      176 non-null    float64
 9   V1real  176 non-null    float64
 10  V2real  176 non-null    float64
 11  V3real  176 non-null    float64
 12  N1      176 non-null    int64  
 13  N2      176 non-null    int64  
 14  N3      176 non-null    int64  
 15  Type    176 non-null    int64  
dtypes: float64(12), int64(4)
memory usage: 22.1 KB


In [4]:
R = 0.04      # 轮子半径，单位：米（40 mm）
L = 0.125     # 轮子到机器人中心的距离（125 mm）
alpha = np.deg2rad(0)  # α：机器人朝向角（单位：弧度）
theta = np.deg2rad(30)  # Θ：轮子朝向角（单位：弧度）

In [5]:
from numpy import cos, sin

# 3x3 矩阵乘以 3x1 向量
M = np.array([
    [(-2/3)*cos(alpha - theta), (2/3)*sin(alpha),       (2/3)*cos(alpha + theta)],
    [(-2/3)*sin(alpha - theta), (-2/3)*cos(alpha),      (2/3)*sin(alpha + theta)],
    [1/(3*L),                   1/(3*L),                1/(3*L)]
])

In [6]:
def compute_velocities(row):
    omega = np.array([row['V1real'], row['V2real'], row['V3real']])
    result = R * (M @ omega)  # R × M × ω 向量
    return pd.Series(result, index=['Vx', 'Vy', 'Omega'])

data[['Vx', 'Vy', 'Omega']] = data.apply(compute_velocities, axis=1)


In [7]:
M_current = np.array([
    [(-2/3)*np.cos(alpha - theta), (2/3)*np.sin(alpha),       (2/3)*np.cos(alpha + theta)],
    [(-2/3)*np.sin(alpha - theta), (-2/3)*np.cos(alpha),      (2/3)*np.sin(alpha + theta)],
    [1/3,                          1/3,                       1/3]
])
# 计算 Ix, Iy, Iphi（逐行应用矩阵乘法）
def compute_projected_currents(row):
    I = np.array([row['I1'], row['I2'], row['I3']])
    result = M_current @ I
    return pd.Series(result, index=['Ix', 'Iy', 'Iphi'])
# IΣ（电流总和）
data['Isigma'] = data['I1'] + data['I2'] + data['I3']


data[['Ix', 'Iy', 'Iphi']] = data.apply(compute_projected_currents, axis=1)

In [8]:
data

,I1,I2,I3,gx,gy,gz,ax,ay,az,V1real,...,N2,N3,Type,Vx,Vy,Omega,Isigma,Ix,Iy,Iphi
0,1.126697,0.021116,0.927601,0.712941,-0.252941,1.081765,0.043529,0.003174,0.011661,-249.352941,...,30,5239,1,11.883906,0.084706,2.202353,2.075414,-0.114948,0.670689,0.691805
1,1.039215,0.015083,0.953243,0.229412,0.276471,0.485294,0.013672,-0.005572,0.012164,-260.470588,...,-9,5391,1,12.103979,0.042353,0.338824,2.007541,-0.049636,0.654097,0.669180
2,0.911011,0.004525,0.947210,1.170588,0.222941,0.664118,0.011704,0.003720,0.005744,-268.411765,...,6,4786,1,12.397408,0.000000,0.000000,1.862746,0.020900,0.616390,0.620915
3,0.692308,1.156862,0.550528,1.165294,0.698235,-3.421765,0.001321,-0.010311,-0.001522,141.352941,...,-5662,2875,1,0.476823,12.218824,-0.338824,2.399698,-0.081857,-0.356963,0.799899
4,0.529412,1.024133,0.453997,0.808235,0.090000,-1.449412,0.006218,-0.010986,-0.006247,155.647059,...,-5824,2967,1,0.330109,12.727059,1.185882,2.007542,-0.043541,-0.354953,0.669181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,0.288889,0.449573,0.270085,0.153333,-2.288667,40.140000,-0.019906,0.003939,-0.000586,349.200000,...,5663,5901,5,0.207846,0.360000,111.744000,1.008547,-0.010856,-0.113390,0.336182
172,0.314530,0.331624,0.341880,0.096000,-2.154667,37.425333,-0.015479,0.000993,-0.009017,347.400000,...,6018,6005,5,0.290985,0.264000,112.128000,0.988034,0.015791,-0.002279,0.329345
173,0.287179,0.377778,0.391453,-1.820667,-5.030000,82.963333,0.027490,0.046403,-0.012874,750.600000,...,12073,12049,5,-0.290985,0.120000,237.696000,1.056410,0.060202,-0.025641,0.352137
174,0.370940,0.427350,0.336752,-0.624000,-4.574000,83.285333,0.014160,-0.043896,0.003906,734.400000,...,12493,12564,5,0.290985,-0.120000,237.504000,1.135042,-0.019738,-0.049003,0.378347


In [9]:
# 为防止除以 0，先加一个小常数 epsilon
epsilon = 1e-6

# 计算 {V3}
data['Tx'] = data['Vx'] / (data['Ix'] + epsilon)
data['Ty'] = data['Vy'] / (data['Iy'] + epsilon)
data['Tphi'] = data['Omega'] / (data['Iphi'] + epsilon)
data['Tz'] = data['gz'] / (data['Iphi'] + epsilon)


In [10]:
data

,I1,I2,I3,gx,gy,gz,ax,ay,az,V1real,...,Vy,Omega,Isigma,Ix,Iy,Iphi,Tx,Ty,Tphi,Tz
0,1.126697,0.021116,0.927601,0.712941,-0.252941,1.081765,0.043529,0.003174,0.011661,-249.352941,...,0.084706,2.202353,2.075414,-0.114948,0.670689,0.691805,-103.385877,0.126297,3.183485,1.563683
1,1.039215,0.015083,0.953243,0.229412,0.276471,0.485294,0.013672,-0.005572,0.012164,-260.470588,...,0.042353,0.338824,2.007541,-0.049636,0.654097,0.669180,-243.859123,0.064750,0.506325,0.725206
2,0.911011,0.004525,0.947210,1.170588,0.222941,0.664118,0.011704,0.003720,0.005744,-268.411765,...,0.000000,0.000000,1.862746,0.020900,0.616390,0.620915,593.161248,0.000000,0.000000,1.069577
3,0.692308,1.156862,0.550528,1.165294,0.698235,-3.421765,0.001321,-0.010311,-0.001522,141.352941,...,12.218824,-0.338824,2.399698,-0.081857,-0.356963,0.799899,-5.825169,-34.230064,-0.423582,-4.277739
4,0.529412,1.024133,0.453997,0.808235,0.090000,-1.449412,0.006218,-0.010986,-0.006247,155.647059,...,12.727059,1.185882,2.007542,-0.043541,-0.354953,0.669181,-7.581752,-35.855742,1.772138,-2.165946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,0.288889,0.449573,0.270085,0.153333,-2.288667,40.140000,-0.019906,0.003939,-0.000586,349.200000,...,0.360000,111.744000,1.008547,-0.010856,-0.113390,0.336182,-19.147176,-3.174901,332.390217,119.399192
172,0.314530,0.331624,0.341880,0.096000,-2.154667,37.425333,-0.015479,0.000993,-0.009017,347.400000,...,0.264000,112.128000,0.988034,0.015791,-0.002279,0.329345,18.426266,-115.883332,340.457044,113.635473
173,0.287179,0.377778,0.391453,-1.820667,-5.030000,82.963333,0.027490,0.046403,-0.012874,750.600000,...,0.120000,237.696000,1.056410,0.060202,-0.025641,0.352137,-4.833364,-4.680160,675.008766,235.599157
174,0.370940,0.427350,0.336752,-0.624000,-4.574000,83.285333,0.014160,-0.043896,0.003906,734.400000,...,-0.120000,237.504000,1.135042,-0.019738,-0.049003,0.378347,-14.742788,2.448889,627.738800,220.128651


In [11]:
# 保存为 Excel 文件，index=False 表示不保存行号
data.to_excel('data/raw/processed_(A+B)_data.xlsx', index=False, engine='openpyxl')

In [12]:
data=pd.read_excel('data/raw/Data_Set_C.xlsx')

In [13]:
data

,I1,I2,I3,gx,gy,gz,ax,ay,az,V1real,V2real,V3real,N1,N2,N3,Type
0,1.143286,0.016591,1.073907,0.343529,-0.032941,0.185294,0.018899,-0.003246,0.004323,-254.117647,0.000000,257.294118,-5081,-2,5062,1
1,0.499246,1.135747,0.481146,0.654118,0.263529,-1.826471,0.009004,-0.015884,-0.007669,162.000000,-308.117647,154.058824,2952,-5933,2991,1
2,0.351433,0.874811,0.942685,0.845882,0.114118,-3.032941,0.016070,-0.034309,-0.008215,-111.176471,-327.176471,462.176471,-2065,-6018,8444,1
3,0.333333,0.459566,0.258382,0.533077,-1.130769,16.233077,-0.008263,0.007343,0.021297,157.846154,162.000000,170.307692,3503,3338,3533,1
4,0.347008,0.353846,0.336752,0.093333,-1.026667,34.422667,-0.003776,-0.000049,0.003906,360.000000,352.800000,349.200000,5917,5931,5939,1
5,0.350961,0.451923,0.437500,-0.688750,-5.436250,76.135000,0.009110,0.022644,0.087982,759.375000,762.750000,767.812500,13176,13081,13128,1
6,0.505983,0.003419,0.288889,0.724000,0.322000,0.722667,-0.001660,0.006966,0.002360,-291.600000,1.800000,300.600000,-4703,6,4877,2
7,0.224359,0.278846,0.174679,0.560625,0.382500,0.042500,-0.009125,-0.019577,0.008163,168.750000,-356.062500,180.562500,3007,-6222,3059,2
8,0.225641,0.283760,0.280342,0.378667,0.157333,1.321333,0.008350,0.009814,0.001465,-113.400000,-354.600000,484.200000,-2035,-6223,8546,2
9,0.282051,0.331624,0.099145,0.541333,-0.878667,20.526667,0.002083,-0.017399,-0.005501,167.400000,165.600000,187.200000,2732,2726,2953,2


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   I1      58 non-null     float64
 1   I2      58 non-null     float64
 2   I3      58 non-null     float64
 3   gx      58 non-null     float64
 4   gy      58 non-null     float64
 5   gz      58 non-null     float64
 6   ax      58 non-null     float64
 7   ay      58 non-null     float64
 8   az      58 non-null     float64
 9   V1real  58 non-null     float64
 10  V2real  58 non-null     float64
 11  V3real  58 non-null     float64
 12  N1      58 non-null     int64  
 13  N2      58 non-null     int64  
 14  N3      58 non-null     int64  
 15  Type    58 non-null     int64  
dtypes: float64(12), int64(4)
memory usage: 7.4 KB


In [15]:
data=pd.read_excel('data/raw/Data_Set_C.xlsx')
R = 0.04      # 轮子半径，单位：米（40 mm）
L = 0.125     # 轮子到机器人中心的距离（125 mm）
alpha = np.deg2rad(0)  # α：机器人朝向角（单位：弧度）
theta = np.deg2rad(30)  # Θ：轮子朝向角（单位：弧度）

from numpy import cos, sin

# 3x3 矩阵乘以 3x1 向量
M = np.array([
    [(-2/3)*cos(alpha - theta), (2/3)*sin(alpha),       (2/3)*cos(alpha + theta)],
    [(-2/3)*sin(alpha - theta), (-2/3)*cos(alpha),      (2/3)*sin(alpha + theta)],
    [1/(3*L),                   1/(3*L),                1/(3*L)]
])

def compute_velocities(row):
    omega = np.array([row['V1real'], row['V2real'], row['V3real']])
    result = R * (M @ omega)  # R × M × ω 向量
    return pd.Series(result, index=['Vx', 'Vy', 'Omega'])

data[['Vx', 'Vy', 'Omega']] = data.apply(compute_velocities, axis=1)

M_current = np.array([
    [(-2/3)*np.cos(alpha - theta), (2/3)*np.sin(alpha),       (2/3)*np.cos(alpha + theta)],
    [(-2/3)*np.sin(alpha - theta), (-2/3)*np.cos(alpha),      (2/3)*np.sin(alpha + theta)],
    [1/3,                          1/3,                       1/3]
])
# 计算 Ix, Iy, Iphi（逐行应用矩阵乘法）
def compute_projected_currents(row):
    I = np.array([row['I1'], row['I2'], row['I3']])
    result = M_current @ I
    return pd.Series(result, index=['Ix', 'Iy', 'Iphi'])
# IΣ（电流总和）
data['Isigma'] = data['I1'] + data['I2'] + data['I3']


data[['Ix', 'Iy', 'Iphi']] = data.apply(compute_projected_currents, axis=1)

data

,I1,I2,I3,gx,gy,gz,ax,ay,az,V1real,...,N2,N3,Type,Vx,Vy,Omega,Isigma,Ix,Iy,Iphi
0,1.143286,0.016591,1.073907,0.343529,-0.032941,0.185294,0.018899,-0.003246,0.004323,-254.117647,...,-2,5062,1,11.810549,4.235294e-02,0.338824,2.233784,-0.040056,0.728004,0.744595
1,0.499246,1.135747,0.481146,0.654118,0.263529,-1.826471,0.009004,-0.015884,-0.007669,162.000000,...,-5933,2991,1,-0.183394,1.243059e+01,0.847059,2.116139,-0.010450,-0.430367,0.705380
2,0.351433,0.874811,0.942685,0.845882,0.114118,-3.032941,0.016070,-0.034309,-0.008215,-111.176471,...,-6018,8444,1,13.241019,1.340471e+01,2.541176,2.168929,0.341360,-0.151835,0.722976
3,0.333333,0.459566,0.258382,0.533077,-1.130769,16.233077,-0.008263,0.007343,0.021297,157.846154,...,3338,3533,1,0.287787,5.538462e-02,52.283077,1.051281,-0.043273,-0.109139,0.350427
4,0.347008,0.353846,0.336752,0.093333,-1.026667,34.422667,-0.003776,-0.000049,0.003906,360.000000,...,5931,5939,1,-0.249415,4.800000e-02,113.280000,1.037606,-0.005922,-0.007977,0.345869
5,0.350961,0.451923,0.437500,-0.688750,-5.436250,76.135000,0.009110,0.022644,0.087982,759.375000,...,13081,13128,1,0.194856,2.250000e-02,244.260000,1.240384,0.049963,-0.038462,0.413461
6,0.505983,0.003419,0.288889,0.724000,0.322000,0.722667,-0.001660,0.006966,0.002360,-291.600000,...,6,4877,2,13.676273,7.200000e-02,1.152000,0.798290,-0.125339,0.262678,0.266097
7,0.224359,0.278846,0.174679,0.560625,0.382500,0.042500,-0.009125,-0.019577,0.008163,168.750000,...,-6222,3059,2,0.272798,1.415250e+01,-0.720000,0.677884,-0.028682,-0.052885,0.225961
8,0.225641,0.283760,0.280342,0.378667,0.157333,1.321333,0.008350,0.009814,0.001465,-113.400000,...,-6223,8546,2,13.800981,1.440000e+01,1.728000,0.789743,0.031581,-0.020513,0.263248
9,0.282051,0.331624,0.099145,0.541333,-0.878667,20.526667,0.002083,-0.017399,-0.005501,167.400000,...,2726,2953,2,0.457261,3.120000e-01,55.488000,0.712820,-0.105601,-0.094017,0.237607


In [16]:
data.to_excel('data/raw/processed_C_data.xlsx', index=False, engine='openpyxl')